In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_theme(style="whitegrid")
np.random.seed(42) 
tamanho_amostra = 100000 


sigma_fixo = 0.5
mus_variaveis = [1.0, 2.0, 3.0]

# Cria uma figura com 3 gráficos lado a lado
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)
fig.suptitle('Efeito da Variação de μ (Centro de Gravidade) com σ Fixo', fontsize=16, y=1.02)

for i, mu in enumerate(mus_variaveis):
    # Gera 100.000 números aleatórios seguindo a distribuição Log-Normal com os parâmetros
    amostra = np.random.lognormal(mean=mu, sigma=sigma_fixo, size=tamanho_amostra)
    
    # Plota o histograma (barras) e a curva de densidade (a "rampa de skate")
    sns.histplot(amostra, bins=100, kde=True, ax=axes[i], line_kws={'linewidth': 3, 'color': 'red'})
    axes[i].set_title(f'μ = {mu:.1f} | σ = {sigma_fixo:.1f}')
    axes[i].set_xlabel('Valor Gerado')
    if i == 0:
        axes[i].set_ylabel('Frequência')

plt.show()


# --- GRÁFICO 2: VARIANDO O PARÂMETRO σ (sigma), O "FATOR ESPALHAMENTO" ---

# Mantemos o mu (centro) fixo para ver apenas o efeito do sigma
mu_fixo = 2.0
sigmas_variaveis = [0.2, 0.7, 1.5]

# Cria uma segunda figura com 3 gráficos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Efeito da Variação de σ (Fator Espalhamento) com μ Fixo', fontsize=16, y=1.02)

for i, sigma in enumerate(sigmas_variaveis):
    # Gera os dados
    amostra = np.random.lognormal(mean=mu_fixo, sigma=sigma, size=tamanho_amostra)
    
    # Plota o gráfico
    sns.histplot(amostra, bins=100, kde=True, ax=axes[i], line_kws={'linewidth': 3, 'color': 'red'})
    axes[i].set_title(f'μ = {mu_fixo:.1f} | σ = {sigma:.1f}')
    axes[i].set_xlabel('Valor Gerado')
    if i == 0:
        axes[i].set_ylabel('Frequência')

plt.show()

In [0]:
# Converte o resultado da sua query Spark para um DataFrame Pandas
df_comparativo_pd = df_comparativo.toPandas()
df_comparativo_pd = df_comparativo_pd.sort_values('mes')

# Inicia a criação do gráfico
fig, ax1 = plt.subplots(figsize=(14, 8))
plt.title('Comparativo de Tendência: Volume de Transações Real vs. Sintético', fontsize=16)

# --- Linha para os Dados REAIS (Eixo da Esquerda) ---
cor_real = 'tab:blue'
ax1.set_xlabel('Mês do Ano')
ax1.set_ylabel('Volume REAL de Transações', color=cor_real, fontsize=12)
ax1.plot(df_comparativo_pd['mes'], df_comparativo_pd['total_tx_pf_pagador_REAL'], color=cor_real, marker='o', label='Volume Real')
ax1.tick_params(axis='y', labelcolor=cor_real)

# --- Linha para os Dados SINTÉTICOS (Eixo da Direita) ---
# Cria um segundo eixo Y que compartilha o mesmo eixo X
ax2 = ax1.twinx()  
cor_sintetico = 'tab:orange'
ax2.set_ylabel('Volume SINTÉTICO de Transações', color=cor_sintetico, fontsize=12)
ax2.plot(df_comparativo_pd['mes'], df_comparativo_pd['total_tx_pf_pagador_SINTETICO'], color=cor_sintetico, marker='x', linestyle='--', label='Volume Sintético')
ax2.tick_params(axis='y', labelcolor=cor_sintetico)

# Adiciona a legenda
fig.legend(loc="upper right", bbox_to_anchor=(0.9, 0.9))
plt.grid(True)
plt.show()

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# --- 1. Geração de Dados de Exemplo ---
# (Você pode pular esta parte e usar seu DataFrame real)
# Nós simulamos o "Ataque de Madrugada" (p_madrugada=0.70)
# e transações legítimas com pico às 14h (loc=14).

print("Gerando dados de exemplo...")

def criar_dados_exemplo(n_legit=50000, n_fraude=5000, p_madrugada=0.7):
    # Gerar horas para transações legítimas (pico em horário comercial)
    horas_legitimas = np.random.normal(loc=14, scale=4.0, size=n_legit)
    # Garantir que as horas estão no intervalo [0, 23]
    horas_legitimas = np.clip(horas_legitimas, 0, 23).astype(int)
    
    # Gerar horas para transações fraudulentas
    n_madrugada = int(n_fraude * p_madrugada)
    n_fraude_normal = n_fraude - n_madrugada
    
    # Fraudes de "Ataque de Madrugada" (1h-4h)
    # np.random.randint(1, 5) gera números inteiros: 1, 2, 3, 4
    horas_fraude_madrugada = np.random.randint(1, 5, size=n_madrugada)
    
    # Outras fraudes que seguem o padrão normal
    horas_fraude_normal = np.random.normal(loc=14, scale=4.0, size=n_fraude_normal)
    horas_fraude_normal = np.clip(horas_fraude_normal, 0, 23).astype(int)
    
    horas_fraude = np.concatenate([horas_fraude_madrugada, horas_fraude_normal])
    
    # Criar DataFrames
    df_legit = pd.DataFrame({'hora': horas_legitimas, 'is_fraud': 0})
    df_fraude = pd.DataFrame({'hora': horas_fraude, 'is_fraud': 1})
    
    # Combinar e embaralhar
    df_transacoes = pd.concat([df_legit, df_fraude]).sample(frac=1).reset_index(drop=True)
    
    # Adicionar uma coluna 'data' apenas para ser fiel ao schema
    # (A hora é a única coisa que importa para este gráfico)
    base_date = datetime(2023, 1, 1)
    df_transacoes['data'] = df_transacoes['hora'].apply(
        lambda h: base_date + timedelta(hours=int(h), minutes=np.random.randint(0, 60))
    )
    
    return df_transacoes


df_transacoes_exemplo = criar_dados_exemplo()

print(df_transacoes_exemplo.head())


df_para_plotar = df_transacoes_exemplo


if 'hora' not in df_para_plotar.columns:
    df_para_plotar['hora'] = df_para_plotar['data'].dt.hour

print("Gerando visualização...")

sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 8))

# - 'x' é a coluna da hora
# - 'hue' é a coluna que separa as cores (fraude vs. legítima)
# - 'bins=24' cria uma barra para cada hora
# - 'multiple="stack"' empilha as barras de fraude sobre as legítimas
viz = sns.histplot(
    data=df_para_plotar,
    x='hora',
    hue='is_fraud',
    bins=24,
    multiple="stack",
    palette={0: "cornflowerblue", 1: "red"}, # 0=Legítima, 1=Fraude
    edgecolor="white",
    linewidth=0.5
)

# **Passo C: Melhorar os rótulos e a legenda**

# Mapear a legenda para nomes mais claros
try:
    handles = viz.legend_.legendHandles
    viz.legend_.remove()
    viz.legend(handles, ['Legítima', 'Fraude'], title='Tipo de Transação')
except AttributeError:
    # Caso a legenda não seja gerada automaticamente
    pass

# Configurar rótulos e título
plt.title('Distribuição de Transações por Hora do Dia (Legítimas vs. Fraudes)', fontsize=16, weight='bold')
plt.xlabel('Hora do Dia (0-23)', fontsize=12)
plt.ylabel('Contagem de Transações', fontsize=12)

# Força a exibição de todas as 24 horas no eixo X
plt.xticks(range(0, 24))
plt.xlim(-0.5, 23.5) # Ajusta os limites para centralizar as barras

# Exibir o gráfico
plt.show()

## Rastreio de Transações Fraudulentas

In [0]:


%sql
select 
 c.id as id_conta_origem,
case
  when startswith (cl.nome, 'Sr. ') then replace (cl.nome,'Sr. ','') 
  when startswith (cl.nome, 'Sra. ') then replace (cl.nome,'Sra. ','')
  when startswith (cl.nome, 'Srta. ') then replace (cl.nome,'Srta. ','')
  when startswith (cl.nome, 'Dra. ') then replace (cl.nome,'Dra. ', '')
  when startswith (cl.nome, 'Dr. ') then replace (cl.nome,'Dr. ','')
  else cl.nome 
  end as nome_remetente, 
  t.valor as valor_enviado,
  t.id_conta_destino,
  case
  when startswith (cl2.nome, 'Sr. ') then replace (cl2.nome,'Sr. ','') 
  when startswith (cl2.nome, 'Sra. ') then replace (cl2.nome,'Sra. ','')
  when startswith (cl2.nome, 'Srta. ') then replace (cl2.nome,'Srta. ','')
  when startswith (cl2.nome, 'Dra. ') then replace (cl2.nome,'Dra. ', '')
  when startswith (cl2.nome, 'Dr. ') then replace (cl2.nome,'Dr. ','')
  else cl2.nome 
  end as nome_destinatario_normalizado,
  t.id as id_transacao,
  t.data,
  t.is_fraud,
  t.fraud_type
from transacoes_db.copper.clientes as cl
join transacoes_db.copper.contas as c
on cl.id = c.id_cliente
join transacoes_db.copper.transacoes as t
on c.id = t.id_conta_origem
join transacoes_db.copper.contas as c2
on t.id_conta_destino = c2.id
join transacoes_db.copper.clientes as cl2 -- join para chamar novamente a nome mas através do relacionamento cl2.id
on c2.id_cliente = cl2.id









In [0]:
%sql
--Não está retornando as contas de destino de algunas fraudadores 
select 
id_conta_destino,
nome
from transacoes_db.copper.transacoes as t
join transacoes_db.copper.contas as c
on t.id_conta_destino = c.id
join transacoes_db.copper.clientes as cli
on c.id_cliente = cli.id
where id_conta_destino in (
'04ab576a-83e6-4b3d-bafc-afcb9cce0eb8')


## Total de Fraudes

In [0]:
%sql
select 
 c.id as id_conta_origem,
case
  when startswith (cl.nome, 'Sr. ') then replace (cl.nome,'Sr. ','') 
  when startswith (cl.nome, 'Sra. ') then replace (cl.nome,'Sra. ','')
  when startswith (cl.nome, 'Srta. ') then replace (cl.nome,'Srta. ','')
  when startswith (cl.nome, 'Dra. ') then replace (cl.nome,'Dra. ', '')
  when startswith (cl.nome, 'Dr. ') then replace (cl.nome,'Dr. ','')
  else cl.nome 
  end as nome_normalizado, 
  t.valor as valor_enviado,
  t.id_conta_destino,
  case
  when startswith (cl2.nome, 'Sr. ') then replace (cl2.nome,'Sr. ','') 
  when startswith (cl2.nome, 'Sra. ') then replace (cl2.nome,'Sra. ','')
  when startswith (cl2.nome, 'Srta. ') then replace (cl2.nome,'Srta. ','')
  when startswith (cl2.nome, 'Dra. ') then replace (cl2.nome,'Dra. ', '')
  when startswith (cl2.nome, 'Dr. ') then replace (cl2.nome,'Dr. ','')
  else cl2.nome 
  end as nome_destinatario_normalizado,
  t.id as id_transacao,
  t.data,
  t.is_fraud,
  t.fraud_type
from transacoes_db.copper.clientes as cl
join transacoes_db.copper.contas as c
on cl.id = c.id_cliente
join transacoes_db.copper.transacoes as t
on c.id = t.id_conta_origem
join transacoes_db.copper.contas as c2
on t.id_conta_destino = c2.id
join transacoes_db.copper.clientes as cl2 -- join para chamar novamente a nome mas através do relacionamento cl2.id
on c2.id_cliente = cl2.id

In [0]:
%sql
select 
c.id_cliente,
c.id as id_conta,
c.agencia,
c.numero,
c.id_tipo_conta,
i.nome as Instituicao_Financeira,
c.estado_ibge,
c.municipio_ibge,
max(c.is_high_risk) as Maior_risco
from transacoes_db.copper.contas as c
left join transacoes_db.copper.instituicoes as i
on c.ispb_instituicao = i.ispb
group by All
order by 
maior_risco desc


Teste


In [0]:
%sql


In [0]:
print({len(df)})

In [0]:
%sql

SELECT
  -- Colunas da transação
  tx.valor AS valor_transacao,
  date(tx.data)  AS data_transacao,
  tx.id_conta_origem AS id_conta_pagador,
  tx.id_conta_destino AS id_conta_recebedor,
  
  tx.id_tipo_iniciacao_pix AS tipo_iniciacao_pix_id,
  tx.id_finalidade_pix AS finalidade_pix_id,
  tx.is_fraud AS transacao_fraudulenta,
  coalesce(tx.fraud_type,'Legitima') AS tipo_fraude,

  -- Pagador: Conta
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  conta_orig.ispb_instituicao AS pagador_ispb_instituicao,
  
  conta_orig.estado_ibge AS pagador_estado_ibge, -- avalaiar se é um campo importante e se é possivel criar uma logica de fraude onde essas informações sejam relevantes
  conta_orig.municipio_ibge AS pagador_municipio_ibge, 

  -- Pagador: Cliente
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.nascido_em AS pagador_data_nascimento,
  cliente_orig.estado_ibge AS pagador_estado_ibge_cliente,
  cliente_orig.municipio_ibge AS pagador_municipio_ibge_cliente,

  -- Pagador: Instituição
  inst_orig.ispb AS pagador_instituicao_ispb,

  -- Pagador: Tipo de Conta
  tipo_conta_orig.id AS pagador_tipo_conta_id_ref,

  -- Pagador: Município
  mun_orig.codigo_ibge AS pagador_municipio_ibge_ref,

  -- Pagador: Natureza
  natureza_orig.id AS pagador_natureza_id_ref,

  -- Recebedor: Conta
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  conta_dest.estado_ibge AS recebedor_estado_ibge, -- avalaiar se é um campo importante e se é possivel criar uma logica de fraude onde essas informações sejam relevantes
  conta_dest.municipio_ibge AS recebedor_municipio_ibge,

  -- Recebedor: Cliente
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.nascido_em AS recebedor_data_nascimento,

  cliente_dest.estado_ibge AS recebedor_estado_ibge_cliente, -- avalaiar se é um campo importante e se é possivel criar uma logica de fraude onde essas informações sejam relevantes
  cliente_dest.municipio_ibge AS recebedor_municipio_ibge_cliente,

  -- Recebedor: Tipo de Conta
  tipo_conta_dest.id AS recebedor_tipo_conta_id_ref,

  -- Recebedor: Município
  mun_dest.codigo_ibge AS recebedor_municipio_ibge_ref,

  -- Recebedor: Natureza
  natureza_dest.id AS recebedor_natureza_id_ref,


-- aumentar o numero de contas de triangulação 
  (
    ROW_NUMBER() OVER (
      PARTITION BY TX.id_conta_origem, tx.id_conta_destino
      ORDER BY   
        tx.data ASC 
    ) - 1

  ) as qtd_transacoes_pagador_recebedor,

    COUNT(tx.id_conta_origem) OVER (
      partition by tx.id_conta_origem, tx.data
    ) as qtd_transacoes_no_dia_pagador,
  
  COUNT(1) OVER (
      PARTITION BY tx.id_conta_destino, date(tx.data)
  ) AS recebedor_total_txs_no_dia
   
FROM
  transacoes_db.copper.transacoes AS tx

 left JOIN transacoes_db.copper.contas AS conta_orig
  ON tx.id_conta_origem = conta_orig.id
 left JOIN transacoes_db.copper.clientes AS cliente_orig
  ON conta_orig.id_cliente = cliente_orig.id
 left JOIN transacoes_db.copper.instituicoes AS inst_orig
  ON conta_orig.ispb_instituicao = inst_orig.ispb
 left JOIN transacoes_db.copper.tipos_conta AS tipo_conta_orig
  ON conta_orig.id_tipo_conta = tipo_conta_orig.id
 left JOIN transacoes_db.copper.municipios AS mun_orig
  ON cliente_orig.municipio_ibge = mun_orig.codigo_ibge
 left JOIN transacoes_db.copper.naturezas AS natureza_orig
  ON cliente_orig.id_natureza = natureza_orig.id

 left JOIN transacoes_db.copper.contas AS conta_dest
  ON tx.id_conta_destino = conta_dest.id
 left JOIN transacoes_db.copper.clientes AS cliente_dest
  ON conta_dest.id_cliente = cliente_dest.id
 left JOIN transacoes_db.copper.instituicoes AS inst_dest
  ON conta_dest.ispb_instituicao = inst_dest.ispb
 left JOIN transacoes_db.copper.tipos_conta AS tipo_conta_dest
  ON conta_dest.id_tipo_conta = tipo_conta_dest.id
 left JOIN transacoes_db.copper.municipios AS mun_dest
  ON cliente_dest.municipio_ibge = mun_dest.codigo_ibge
 left JOIN transacoes_db.copper.naturezas AS natureza_dest
  ON cliente_dest.id_natureza = natureza_dest.id

 left JOIN transacoes_db.copper.finalidade_pix AS finalidade_pix
  ON tx.id_finalidade_pix = finalidade_pix.id

where tx.is_fraud = 1
ORDER BY
  recebedor_total_txs_no_dia DESC

In [0]:
%sql
SELECT
  *
FROM
  system.access.table_lineage
WHERE
  target_table_full_name = 'transacoes_db.gold.transacoes_dataset'

In [0]:
%sql


WITH colunas AS (
    SELECT
        c.TABLE_CATALOG,
        c.TABLE_SCHEMA,
        c.TABLE_NAME,
        c.COLUMN_NAME,
        c.DATA_TYPE,
        c.IS_NULLABLE,
        c.ORDINAL_POSITION
    FROM transacoes_db.information_schema.columns c
    UNION ALL
    SELECT
        c.TABLE_CATALOG,
        c.TABLE_SCHEMA,
        c.TABLE_NAME,
        c.COLUMN_NAME,
        c.DATA_TYPE,
        c.IS_NULLABLE,
        c.ORDINAL_POSITION
    FROM estatisticas_pix.information_schema.columns c
),

tabelas AS (
    SELECT
        t.TABLE_CATALOG,
        t.TABLE_SCHEMA,
        t.TABLE_NAME,
        t.TABLE_TYPE,
        t.TABLE_OWNER,
        t.COMMENT
    FROM transacoes_db.information_schema.tables t
    UNION ALL
    SELECT
        t.TABLE_CATALOG,
        t.TABLE_SCHEMA,
        t.TABLE_NAME,
        t.TABLE_TYPE,
        t.TABLE_OWNER,
        t.COMMENT
    FROM estatisticas_pix.information_schema.tables t
),

-- 🔹 Agrega a linhagem para cada tabela de destino
linhagem_agregada AS (
    SELECT
        target_table_full_name,
        collect_set(source_table_full_name) AS Tabelas_Origem,
        max(event_time) AS Linhagem_Ultima_Atualizacao_UTC
    FROM system.access.table_lineage
    WHERE target_table_catalog IN ('transacoes_db', 'estatisticas_pix')
    GROUP BY target_table_full_name
)

SELECT
    -- Identificação da tabela
    T.TABLE_CATALOG AS Catalogo_Destino,
    T.TABLE_SCHEMA AS Esquema_Destino,
    T.TABLE_NAME AS Nome_Tabela_Destino,
    T.TABLE_TYPE AS Tipo_Objeto,

    -- Detalhes da coluna
    C.COLUMN_NAME AS Nome_Coluna,
    C.DATA_TYPE AS Tipo_Dado,
    C.IS_NULLABLE AS Permite_Nulo,
    C.ORDINAL_POSITION AS Posicao,

    -- Linhagem (origens agregadas)
    L.Tabelas_Origem,
    L.Linhagem_Ultima_Atualizacao_UTC,

    -- Metadados adicionais
    T.TABLE_OWNER AS Proprietario_Tabela,
    T.COMMENT AS Comentario_Tabela

FROM colunas C
JOIN tabelas T
    ON C.TABLE_CATALOG = T.TABLE_CATALOG
    AND C.TABLE_SCHEMA = T.TABLE_SCHEMA
    AND C.TABLE_NAME = T.TABLE_NAME
LEFT JOIN linhagem_agregada L
    ON CONCAT(T.TABLE_CATALOG, '.', T.TABLE_SCHEMA, '.', T.TABLE_NAME) = L.target_table_full_name

WHERE
    T.TABLE_CATALOG IN ('transacoes_db', 'estatisticas_pix')
    AND T.TABLE_TYPE IN ('MANAGED', 'EXTERNAL', 'VIEW') 
    and t.TABLE_SCHEMA <> "information_schema"

ORDER BY
    Catalogo_Destino,
    Esquema_Destino,
    Nome_Tabela_Destino,
    Posicao;


In [0]:
%sql
SELECT
    -- Identificação da Tabela
    T.TABLE_CATALOG AS Catalogo,
    T.TABLE_SCHEMA AS Esquema,
    T.TABLE_NAME AS Nome_Tabela,
    T.TABLE_TYPE AS Tipo_Objeto,
    
    -- Detalhes da Coluna
    C.COLUMN_NAME AS Nome_Coluna,
    C.DATA_TYPE AS Tipo_Dado,
    C.IS_NULLABLE AS Permite_Nulo,
    C.ORDINAL_POSITION AS Posicao,
    
    -- Metadados Adicionais
    T.TABLE_OWNER AS Proprietario_Tabela,
    T.COMMENT AS Comentario_Tabela
    
FROM
    -- Consulta a tabela de metadados do sistema (Unity Catalog)
    system.information_schema.tables T
JOIN
    system.information_schema.columns C
    ON T.TABLE_CATALOG = C.TABLE_CATALOG
    AND T.TABLE_SCHEMA = C.TABLE_SCHEMA
    AND T.TABLE_NAME = C.TABLE_NAME
WHERE
    T.TABLE_CATALOG = 'transacoes_db'
    AND T.TABLE_TYPE IN ('MANAGED', 'EXTERNAL') 
ORDER BY
    Esquema,
    Nome_Tabela,
    Posicao;

In [0]:
%sql

WITH tx_com_long AS (
  SELECT
    *,
    CAST(tx.data AS LONG) AS data_em_segundos
  FROM
    transacoes_db.gold.transacoes_balanced_model AS tx -- <<< 1. MUDANÇA AQUI (Usando a tabela V8)
),

tx_features_realtime AS (
  SELECT
    tx.*,
    
    (COUNT(1) OVER (
      PARTITION BY id_conta_pagador
      ORDER BY data_em_segundos
      RANGE BETWEEN (24 * 3600) PRECEDING AND 1 PRECEDING 
    )) AS pagador_txs_ultimas_24h,
    
    (COALESCE(SUM(valor_transacao) OVER (
      PARTITION BY id_conta_pagador
      ORDER BY data_em_segundos
      RANGE BETWEEN (24 * 3600) PRECEDING AND 1 PRECEDING
    ), 0)) AS pagador_valor_ultimas_24h,

  
    (COUNT(1) OVER (
      PARTITION BY id_conta_recebedor
      ORDER BY data_em_segundos
      RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING 
    )) AS recebedor_txs_ultima_1h,
    
    (COALESCE(SUM(valor_transacao) OVER (
      PARTITION BY id_conta_recebedor
      ORDER BY data_em_segundos
      RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING
    ), 0)) AS recebedor_valor_ultima_1h,

    (
      data_em_segundos - LAG(data_em_segundos, 1, 0) OVER (
        PARTITION BY id_conta_pagador
        ORDER BY data_em_segundos
      )
    ) AS pagador_segundos_desde_ultima_tx

  FROM
    tx_com_long tx
),

SELECT
  -- Colunas da transação
  ft.valor_transacao,
  ft.data_transacao,
  ft.id_conta_pagador,
  ft.id_conta_recebedor,
  ft.tipo_iniciacao_pix_id,
  ft.finalidade_pix_id,
  ft.transacao_fraudulenta,
  ft.tipo_fraude,

  -- Pagador: Perfil
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.nascido_em AS pagador_data_nascimento,
  
  -- Recebedor: Perfil
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.nascido_em AS recebedor_data_nascimento,

  ft.pagador_txs_ultimas_24h,
  ft.pagador_valor_ultimas_24h,
  ft.recebedor_txs_ultima_1h,
  ft.recebedor_valor_ultima_1h,
  ft.pagador_segundos_desde_ultima_tx


FROM
  tx_features_realtime AS ft
  
  LEFT JOIN transacoes_db.copper.contas AS conta_orig
    ON ft.id_conta_pagador = conta_orig.id
  LEFT JOIN transacoes_db.copper.clientes AS cliente_orig
    ON conta_orig.id_cliente = cliente_orig.id
  LEFT JOIN transacoes_db.copper.contas AS conta_dest
    ON ft.id_conta_recebedor = conta_dest.id
  LEFT JOIN transacoes_db.copper.clientes AS cliente_dest
    ON conta_dest.id_cliente = cliente_dest.id


In [0]:

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pyspark.sql.functions as F

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

df_transacoes = spark.table("transacoes_db.copper.transacoes")
df_clientes = spark.table("transacoes_db.copper.clientes")
df_contas = spark.table("transacoes_db.copper.contas")

print("Tabelas carregadas com sucesso.")

df_hora = df_transacoes.select(
    F.hour("data").alias("hora"), 
    F.col("is_fraud").cast("string")
).sample(fraction=0.1, seed=42).toPandas()

plt.figure(figsize=(12, 6))
sns.histplot(
    data=df_hora, 
    x="hora", 
    hue="is_fraud", 
    multiple="dodge", 
    bins=24, 
    palette={ "0": "blue", "1": "red" }
)
plt.title("Distribuição de Transações por Hora: Legítimas (0) vs. Fraudes (1)")
plt.xlabel("Hora do Dia")
plt.ylabel("Contagem")
plt.show()


df_vitimas = df_transacoes.filter(F.col("fraud_type") == "engenharia_social") \
    .join(df_contas.alias("c"), F.col("id_conta_origem") == F.col("c.id")) \
    .join(df_clientes.alias("cli"), F.col("c.id_cliente") == F.col("cli.id")) \
    .withColumn("idade", F.floor(F.datediff(F.current_date(), F.col("cli.nascido_em")) / 365.25)) \
    .select("idade") \
    .toPandas()

plt.figure(figsize=(10, 5))
sns.histplot(df_vitimas["idade"], kde=True, color="orange", bins=20)
plt.axvline(55, color='r', linestyle='--', label='Corte do Algoritmo (55+)')
plt.title("Distribuição de Idade das Vítimas de Engenharia Social")
plt.xlabel("Idade")
plt.legend()
plt.show()

df_radar = df_transacoes.filter(
    (F.col("is_fraud") == 1) & 
    (F.col("valor").between(400, 2100))
).select("valor").toPandas()

plt.figure(figsize=(12, 6))
# Usamos muitos bins (200) para conseguir visualizar os picos finos
sns.histplot(df_radar["valor"], bins=200, color="purple")
plt.title("Frequência de Valores em Fraudes (Zoom R$ 400 - R$ 2100)")
plt.xlabel("Valor da Transação")
plt.ylabel("Frequência")

# Anotações para destacar os picos esperados
plt.annotate('Pico 499.90', xy=(499.90, 10), xytext=(499.90, 100), 
             arrowprops=dict(facecolor='black', shrink=0.05))
plt.annotate('Pico 999.90', xy=(999.90, 10), xytext=(999.90, 100), 
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()


df_tipos = df_transacoes.filter(F.col("is_fraud") == 1) \
    .select("fraud_type", "valor").toPandas()

plt.figure(figsize=(14, 6))
sns.boxplot(x="fraud_type", y="valor", data=df_tipos, showfliers=False) 
plt.title("Distribuição de Valores por Tipo de Fraude")
plt.yscale("log") 
plt.ylabel("Valor (R$) - Escala Log")
plt.xlabel("Tipo de Fraude")
plt.xticks(rotation=15) 
plt.show()

In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. Papel das Contas de Risco ("Laranjas")
# MAGIC Analisa a proporção de transações enviadas para contas marcadas como `is_high_risk` (Laranjas) para cada tipo de fraude.
# MAGIC
# MAGIC **Expectativa:** Fraudes complexas (Triangulação) e Engenharia Social devem ter alta concentração em contas de risco, enquanto transações legítimas devem ter baixa concentração.

# COMMAND ----------

# 1. Faz o Join da transação com a conta de DESTINO para ver o flag de risco
df_risco_destino = df_transacoes.alias("t") \
    .join(df_contas.alias("c"), F.col("t.id_conta_destino") == F.col("c.id"), "inner") \
    .select(
        F.col("t.fraud_type").alias("Tipo de Transação"),
        F.col("c.is_high_risk").alias("Conta Destino de Risco")
    ) \
    .fillna({"Tipo de Transação": "Legítima"}) \
    .toPandas()

# 2. Prepara os dados para o gráfico (Crosstab normalizado)
crosstab = pd.crosstab(
    df_risco_destino["Tipo de Transação"], 
    df_risco_destino["Conta Destino de Risco"], 
    normalize='index' # Normaliza para mostrar percentual (0 a 1)
) * 100

# Ordenar para deixar "Legítima" na ponta ou base
crosstab = crosstab.sort_values(by=1, ascending=True)

# 3. Plotagem (Gráfico de Barras Empilhadas Horizontal)
ax = crosstab.plot(kind='barh', stacked=True, color={0: "#a8dadc", 1: "#e63946"}, figsize=(12, 6))

plt.title("Proporção de Contas Destino: Normal (0) vs. Alto Risco/Laranja (1)")
plt.xlabel("Porcentagem (%)")
plt.ylabel("Tipo de Transação")
plt.legend(title="Destino é Alto Risco?", loc="lower right", labels=["Não (Normal)", "Sim (Laranja)"])


for n, x in enumerate([*crosstab.index.values]):
    valor_risco = crosstab.loc[x, 1]
    if valor_risco > 5: 
        plt.text(x=100 - (valor_risco/2), y=n, s=f"{valor_risco:.1f}%", 
                 va='center', ha='center', color='white', fontweight='bold')

plt.xlim(0, 100)
plt.tight_layout()
plt.show()

In [0]:
!pip install networkx

In [0]:
import networkx as nx

id_fraude_exemplo = df_transacoes.filter(
    (F.col("fraud_type") == "triangulacao_conta_laranja") & 
    (F.col("id_transacao_cadeia_pai").isNotNull())
).select("id_transacao_cadeia_pai").limit(1).collect()[0][0]

df_cadeia = df_transacoes.filter(
    (F.col("id") == id_fraude_exemplo) | 
    (F.col("id_transacao_cadeia_pai") == id_fraude_exemplo)
).select("id_conta_origem", "id_conta_destino", "valor", "mensagem").toPandas()

G = nx.DiGraph()

for _, row in df_cadeia.iterrows():

    G.add_edge(row['id_conta_origem'][-4:], row['id_conta_destino'][-4:], weight=row['valor'])


plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42) 


nx.draw_networkx_nodes(G, pos, node_size=700, node_color="#e63946", alpha=0.9)

nx.draw_networkx_edges(G, pos, width=2, alpha=0.6, edge_color="gray", arrowsize=20)

nx.draw_networkx_labels(G, pos, font_size=10, font_color="white", font_weight="bold")

edge_labels = nx.get_edge_attributes(G, 'weight')
edge_labels_formatado = {k: f"R$ {v:.0f}" for k, v in edge_labels.items()}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels_formatado, font_size=8)

plt.title(f"Topologia de Rede: Exemplo de Triangulação (ID Raiz: ...{id_fraude_exemplo[-6:]})")
plt.axis('off')
plt.show()

In [0]:

df_idade_chave = df_transacoes.join(
    df_contas.alias("c"), 
    F.col("id_conta_destino") == F.col("c.id")
).join(
    spark.table("transacoes_db.copper.chaves_pix").alias("k"),
    F.col("c.id") == F.col("k.id_conta")
).select(
    F.col("is_fraud").cast("string"),
    F.datediff(F.col("data"), F.col("k.cadastrada_em")).alias("dias_desde_cadastro")
).filter(F.col("dias_desde_cadastro") >= 0) # Filtra inconsistências de data


df_idade_pd = df_idade_chave.sample(fraction=0.2, seed=42).toPandas()


plt.figure(figsize=(12, 6))


sns.kdeplot(
    data=df_idade_pd[df_idade_pd['dias_desde_cadastro'] <= 100], 
    x="dias_desde_cadastro", 
    hue="is_fraud", 
    fill=True, 
    common_norm=False, 
    palette={ "0": "green", "1": "red" },
    alpha=0.3
)

plt.axvline(15, color='black', linestyle='--', label='Limiar de Risco (15 dias)')
plt.title("Densidade de Transações por Idade da Chave PIX")
plt.xlabel("Dias desde o cadastro da Chave PIX")
plt.ylabel("Densidade")
plt.legend(title="É Fraude?")
plt.show()

In [0]:


df_geo = df_transacoes.select("id_conta_origem", "id_conta_destino") \
    .join(df_contas.select(F.col("id").alias("ido"), F.col("estado_ibge").alias("uf_origem")), 
          F.col("id_conta_origem") == F.col("ido")) \
    .join(df_contas.select(F.col("id").alias("idd"), F.col("estado_ibge").alias("uf_destino")), 
          F.col("id_conta_destino") == F.col("idd")) \
    .select("uf_origem", "uf_destino") \
    .sample(fraction=0.1, seed=42).toPandas() # Amostra para performance

# 2. Criar Matriz (Crosstab)
matriz_geo = pd.crosstab(df_geo['uf_origem'], df_geo['uf_destino'])

# 3. Plotagem (Heatmap)
plt.figure(figsize=(12, 10))
sns.heatmap(
    matriz_geo, 
    annot=True, 
    fmt='d', 
    cmap="YlGnBu", 
    cbar_kws={'label': 'Número de Transações'}
)

plt.title("Matriz de Origem x Destino (Estados IBGE)")
plt.xlabel("Estado de Destino")
plt.ylabel("Estado de Origem")
plt.show()

In [0]:

df_saldo_valor = df_transacoes.alias("t") \
    .join(df_contas.alias("c"), F.col("t.id_conta_origem") == F.col("c.id")) \
    .select(
        F.col("t.valor"),
        F.col("c.saldo"),
        F.col("t.is_fraud").cast("string")
    ).sample(fraction=0.1, seed=123).toPandas() # Amostra para não travar o plot

# 2. Plotagem (Scatterplot)
plt.figure(figsize=(12, 8))

sns.scatterplot(
    data=df_saldo_valor, 
    x="saldo", 
    y="valor", 
    hue="is_fraud", 
    alpha=0.6,
    palette={ "0": "#2a9d8f", "1": "#e63946" },
    style="is_fraud"
)

# Linha de referência (Valor = Saldo)
plt.plot([0, df_saldo_valor['saldo'].max()], [0, df_saldo_valor['saldo'].max()], 
         color='gray', linestyle='--', label='Linha de Equilíbrio (Valor = Saldo)')

plt.title("Dispersão: Valor da Transação vs. Saldo em Conta")
plt.xlabel("Saldo da Conta de Origem (R$)")
plt.ylabel("Valor da Transação (R$)")
plt.xscale("log") # Log para visualizar melhor as ordens de magnitude
plt.yscale("log")
plt.legend(title="É Fraude?")
plt.show()

In [0]:

df_sazonal = df_transacoes.select(F.dayofmonth("data").alias("dia_mes")) \
    .groupBy("dia_mes") \
    .count() \
    .orderBy("dia_mes") \
    .toPandas()

# 2. Plotagem (Lineplot)
plt.figure(figsize=(14, 5))
sns.lineplot(data=df_sazonal, x="dia_mes", y="count", marker="o", color="#264653", linewidth=2.5)

# Destacar dias de pico esperados no código (ex: 5, 10, 20)
dias_pico = [5, 10, 20]
for dia in dias_pico:
    valor = df_sazonal.loc[df_sazonal['dia_mes'] == dia, 'count'].values[0] if dia in df_sazonal['dia_mes'].values else 0
    if valor > 0:
        plt.annotate(f'Dia {dia}', xy=(dia, valor), xytext=(dia, valor*1.1),
                     arrowprops=dict(facecolor='red', shrink=0.05), ha='center')

plt.title("Volume Diário de Transações (Padrão Mensal Agregado)")
plt.xlabel("Dia do Mês")
plt.ylabel("Total de Transações")
plt.xticks(range(1, 32))
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [0]:

# 1. Agregação
df_iniciacao = df_transacoes.groupBy("id_tipo_iniciacao_pix").agg(
    F.count("*").alias("total"),
    F.sum("is_fraud").alias("fraudes")
).toPandas()

# Calcular taxa
df_iniciacao["taxa_fraude"] = (df_iniciacao["fraudes"] / df_iniciacao["total"]) * 100

# Mapeamento (Exemplo hipotético de IDs, ajustar conforme dicionário real do sistema)
mapa_ids = {1: "Chave Dict", 2: "Manual", 3: "QR Code Estático", 4: "QR Code Dinâmico"}
df_iniciacao["tipo_nome"] = df_iniciacao["id_tipo_iniciacao_pix"].map(mapa_ids)

# 2. Plotagem (Barplot Duplo)
fig, ax1 = plt.subplots(figsize=(12, 6))

# Barras de Volume
sns.barplot(data=df_iniciacao, x="tipo_nome", y="total", color="lightgray", ax=ax1, label="Volume Total")
ax1.set_ylabel("Volume Total de Transações")

# Linha de Taxa de Fraude
ax2 = ax1.twinx()
sns.lineplot(data=df_iniciacao, x="tipo_nome", y="taxa_fraude", color="red", marker="o", linewidth=3, ax=ax2, label="Taxa de Fraude (%)")
ax2.set_ylabel("Taxa de Fraude (%)", color="red")
ax2.tick_params(axis='y', labelcolor="red")

plt.title("Volume vs. Taxa de Fraude por Tipo de Iniciação")
plt.show()

In [0]:


# 1. Preparação dos dados (Agregado por Mês e Dia)
df_calendario = df_transacoes.select(
    F.month("data").alias("mes"),
    F.dayofmonth("data").alias("dia")
).groupBy("mes", "dia").count().toPandas()

# 2. Pivotar para formato de matriz (Mes nas linhas, Dias nas colunas)
matriz_calendario = df_calendario.pivot(index='mes', columns='dia', values='count')
# Ordenar meses
matriz_calendario = matriz_calendario.sort_index()

# 3. Plotagem (Heatmap)
plt.figure(figsize=(20, 8))
ax = sns.heatmap(
    matriz_calendario, 
    cmap="YlOrRd", 
    linewidths=.5, 
    annot=False, # False para não poluir, use True se quiser ver os números
    cbar_kws={'label': 'Volume de Transações'}
)

plt.title("Mapa de Calor de Sazonalidade: O Ano em Pix")
plt.xlabel("Dia do Mês")
plt.ylabel("Mês")
plt.yticks(ticks=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5], 
           labels=['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'], 
           rotation=0)

plt.show()

In [0]:


import numpy as np

# 1. Extrair o primeiro dígito (excluindo zero e nulos)
# Convertemos para string, pegamos o primeiro char. Se for 0 (ex: 0.50), pegamos o segundo se não for ponto.
# Simplificação: Pegamos o primeiro digito não-zero da esquerda.
df_benford = df_transacoes.filter(F.col("valor") >= 1.0) \
    .select(F.substring(F.col("valor").cast("string"), 1, 1).alias("primeiro_digito")) \
    .filter(F.col("primeiro_digito").isin([str(x) for x in range(1, 10)])) \
    .groupBy("primeiro_digito") \
    .count() \
    .orderBy("primeiro_digito") \
    .toPandas()

# 2. Calcular frequências
total = df_benford['count'].sum()
df_benford['freq_real'] = df_benford['count'] / total

# 3. Curva Teórica de Benford
digitos = np.arange(1, 10)
benford_teorico = np.log10(1 + 1/digitos)

# 4. Plotagem
plt.figure(figsize=(12, 6))

# Barras (Dados Reais)
sns.barplot(x=df_benford['primeiro_digito'], y=df_benford['freq_real'], color="skyblue", label="Dados Sintéticos (IARA)")

# Linha (Teoria)
plt.plot(range(9), benford_teorico, color='red', marker='o', linestyle='--', linewidth=2, label="Lei de Benford (Teórico)")

plt.title("Teste de Realismo Financeiro: Lei de Benford")
plt.xlabel("Primeiro Dígito do Valor da Transação")
plt.ylabel("Frequência")
plt.legend()
plt.show()

In [0]:


# 1. Calcular Out-Degree (Quantas vezes foi origem)
df_out = df_transacoes.groupBy("id_conta_origem").count().withColumnRenamed("count", "out_degree")

# 2. Calcular In-Degree (Quantas vezes foi destino)
df_in = df_transacoes.groupBy("id_conta_destino").count().withColumnRenamed("count", "in_degree")

# 3. Join e preenchimento de nulos (quem só recebeu ou só enviou)
# Usamos full outer join para pegar todas as contas ativas
df_graus = df_out.join(df_in, df_out.id_conta_origem == df_in.id_conta_destino, "full_outer") \
    .select(
        F.coalesce(F.col("id_conta_origem"), F.col("id_conta_destino")).alias("id_conta"),
        F.coalesce(F.col("out_degree"), F.lit(0)).alias("out_degree"),
        F.coalesce(F.col("in_degree"), F.lit(0)).alias("in_degree")
    )

# 4. Enriquecer com flag se a conta já foi destino de fraude (para colorir o gráfico)
contas_fraude_destino = df_transacoes.filter(F.col("is_fraud") == 1).select("id_conta_destino").distinct()
df_plot_graus = df_graus.join(contas_fraude_destino, df_graus.id_conta == contas_fraude_destino.id_conta_destino, "left") \
    .withColumn("envolvida_fraude", F.when(F.col("id_conta_destino").isNotNull(), "Sim").otherwise("Não")) \
    .sample(fraction=0.05, seed=42).toPandas() # Amostra para scatter

# 5. Plotagem
plt.figure(figsize=(10, 10))
sns.scatterplot(
    data=df_plot_graus, 
    x="out_degree", 
    y="in_degree", 
    hue="envolvida_fraude", 
    style="envolvida_fraude",
    alpha=0.6,
    palette={"Não": "gray", "Sim": "red"},
    s=60
)

# Linha de equilíbrio
max_val = max(df_plot_graus['out_degree'].max(), df_plot_graus['in_degree'].max())
plt.plot([0, max_val], [0, max_val], '--', color='black', alpha=0.3, label="Comportamento Equilibrado")

plt.title("Dispersão de Fluxo: Grau de Entrada vs. Saída")
plt.xlabel("Grau de Saída (Transações Enviadas)")
plt.ylabel("Grau de Entrada (Transações Recebidas)")
plt.legend(title="Destino de Fraude?")
plt.show()

In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 16. Padrão Comportamental: "Testing the Waters"
# MAGIC Verifica a existência de micro-transações (testes) que precedem imediatamente grandes transferências fraudulentas entre as mesmas partes.
# MAGIC
# MAGIC **Expectativa:** Identificar pares (Origem->Destino) que possuem uma transação < R$ 2,00 seguida por uma transação > R$ 1000,00 em menos de 10 minutos.

# COMMAND ----------

from pyspark.sql.window import Window

# 1. Definir janela por par de contas, ordenada por tempo
w = Window.partitionBy("id_conta_origem", "id_conta_destino").orderBy("data")

# 2. Calcular Lag (Valor e Data da transação anterior)
df_lag = df_transacoes.withColumn("valor_anterior", F.lag("valor").over(w)) \
                      .withColumn("data_anterior", F.lag("data").over(w)) \
                      .withColumn("is_fraud_anterior", F.lag("is_fraud").over(w))

# 3. Filtrar o padrão "Teste -> Ataque"
# Regra: Anterior < 2.00, Atual > 500.00, Diferença de tempo < 15 minutos, Atual é Fraude
df_padrao_teste = df_lag.filter(
    (F.col("is_fraud") == 1) &
    (F.col("valor") > 500) &
    (F.col("valor_anterior") < 2.00) &
    ((F.col("data").cast("long") - F.col("data_anterior").cast("long")) < 900) # 900 segundos = 15 min
).select(
    "id_conta_origem", "id_conta_destino", 
    F.col("valor_anterior").alias("Valor Teste"), 
    F.col("valor").alias("Valor Ataque"),
    F.col("data_anterior").alias("Hora Teste"),
    F.col("data").alias("Hora Ataque")
).limit(10).toPandas()

display(df_padrao_teste)